In [1]:
import requests
import json
from pprint import pprint

from digital_hospitals.bim import models
from digital_hospitals.bim.app import BimResult

In [2]:
def pprint_response(r: requests.Response):
    print(f'{r.request.method} {r.request.path_url}')
    print()

    print('----REQUEST----HEADERS-------------------------------')
    pprint(dict(r.request.headers))
    print()

    print('----RESPONSE----HEADERS------------------------------')
    pprint(dict(r.headers))
    print()
    print(f'----RESPONSE----CONTENT----{r.status_code}-----------------------')
    print(r.encoding)
    print()
    if ('text/' in r.headers['Content-Type']):
        print(r.text)
    elif ('application/json' in r.headers['Content-Type']):
        pprint(r.json())
    else:
        print(r.raw)

In [3]:
response = requests.get(url='http://localhost/api/example/', params={'test': 'ignore this'})
pprint_response(response)

GET /api/example/?test=ignore+this

----REQUEST----HEADERS-------------------------------
{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Connection': 'keep-alive',
 'User-Agent': 'python-requests/2.32.3'}

----RESPONSE----HEADERS------------------------------
{'Connection': 'keep-alive',
 'Content-Length': '11',
 'Content-Type': 'text/plain; charset=utf-8',
 'Date': 'Tue, 04 Jun 2024 18:08:45 GMT',
 'Server': 'nginx/1.25.5'}

----RESPONSE----CONTENT----200-----------------------
utf-8

Hello World


In [4]:
response = requests.get(url='http://localhost/api/example/notimplemented')
pprint_response(response)

GET /api/example/notimplemented

----REQUEST----HEADERS-------------------------------
{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Connection': 'keep-alive',
 'User-Agent': 'python-requests/2.32.3'}

----RESPONSE----HEADERS------------------------------
{'Connection': 'keep-alive',
 'Content-Length': '28',
 'Content-Type': 'application/json',
 'Date': 'Tue, 04 Jun 2024 18:08:45 GMT',
 'Server': 'nginx/1.25.5'}

----RESPONSE----CONTENT----501-----------------------
utf-8

{'detail': 'Not implemented'}


In [5]:
door_list = [f'd{n}' for n in range(1,17)]
extra_paths = [
    models.Path(path=['d10', 'd12'], duration_seconds=120., required_assets=['Lift']).model_dump(),
    models.Path(path=['d11', 'd13'], duration_seconds=45., required_assets=['Lift']).model_dump()
]

params = {'door_list': door_list, 'extra_paths': extra_paths}

In [6]:
with open('../secrets/histo.ifc', 'rb') as ifc_file:
    response = requests.post(
        url='http://localhost/api/bim/',
        files={'file': ifc_file},
        data={'form_data': json.dumps(params)}
    )

pprint_response(response)

POST /api/bim/

----REQUEST----HEADERS-------------------------------
{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Connection': 'keep-alive',
 'Content-Length': '18020965',
 'Content-Type': 'multipart/form-data; '
                 'boundary=2293dd9bed3e86e8602936f45f7dd320',
 'User-Agent': 'python-requests/2.32.3'}

----RESPONSE----HEADERS------------------------------
{'Connection': 'keep-alive',
 'Content-Length': '53',
 'Content-Type': 'application/json',
 'Date': 'Tue, 04 Jun 2024 18:08:50 GMT',
 'Server': 'nginx/1.25.5'}

----RESPONSE----CONTENT----202-----------------------
utf-8

{'detail': 'Accepted', 'id': '665f58328020df4badea1726'}


In [7]:
my_id = json.loads(response.content)['id']

In [8]:
response = requests.get(
    url='http://localhost/api/bim/query',
    params={'id': my_id},
)

pprint_response(response)

GET /api/bim/query?id=665f58328020df4badea1726

----REQUEST----HEADERS-------------------------------
{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Connection': 'keep-alive',
 'User-Agent': 'python-requests/2.32.3'}

----RESPONSE----HEADERS------------------------------
{'Connection': 'keep-alive',
 'Content-Length': '81',
 'Content-Type': 'application/json',
 'Date': 'Tue, 04 Jun 2024 18:08:50 GMT',
 'Server': 'nginx/1.25.5'}

----RESPONSE----CONTENT----200-----------------------
utf-8

{'err_msg': None,
 'graph': None,
 'requested_ts': 1717524530.273542,
 'status': 'Running'}


In [9]:
BimResult.model_validate(response.json()).status

'Running'